In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [30]:
url_inicial = 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/archive'
url_root = 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/archive'
r=requests.get(url_inicial)

In [31]:
soup = BeautifulSoup(r.text, 'html.parser')

In [32]:
box = soup.find('div', class_='page page_issue_archive')
volumen=box.findAll('div',class_='issue-block news-block col-sm-6 col-md-6 col-lg-4')
vol = [x.find('a').get('href')for x in volumen]
vol

['https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/69',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/68',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/67',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/66',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/65',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/63',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/62',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/61',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/60',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/58',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/57',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/56',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/53',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/vi

In [33]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='container') 
    volumen1=box1.findAll('div', class_='card one-article-intoc')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [34]:
vol2

['https://perfilesla.flacso.edu.mx/index.php/perfilesla/article/view/1410',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/article/view/1470',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/article/view/1490',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/article/view/1509',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/article/view/1471',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/article/view/1418',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/article/view/1492',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/article/view/1511',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/article/view/1546',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/article/view/1428',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/article/view/1512',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/article/view/1518',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/article/view/1497',
 'https://pe

In [35]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='container') 
    volumen1=box1.findAll('div', class_='card one-article-intoc')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [36]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/archive


In [37]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

355

In [38]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [39]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Perfiles Latinoamericanos'
    a='V. Ciencias Sociales'
    tem='Ciencia Política; Sociología; Economía, Políticas Públicas; Educación; Población y Desarrollo.'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='article-fulltext').get_text(strip=True)
        content_book['Resumen']=resu.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='galley-link obj_galley_link pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [ ]:
list_scraper=list_scraper[0:355]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [26]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...","Plebiscitos, referendos e iniciativas populare...",En este artículo se estudian y ordenan las dif...,None,https://perfilesla.flacso.edu.mx/index.php/per...,https://perfilesla.flacso.edu.mx/index.php/per...
1,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",Mejorando los programas de combate a la pobrez...,Muchos de los programas sociales y la gran may...,None,https://perfilesla.flacso.edu.mx/index.php/per...,https://perfilesla.flacso.edu.mx/index.php/per...
2,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",Rearme y baja percepción de amenaza interestat...,El propósito de este artículo es realizar un a...,None,https://perfilesla.flacso.edu.mx/index.php/per...,https://perfilesla.flacso.edu.mx/index.php/per...
3,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",Evaluación de la reforma del sistema de pensio...,En el presente artículo se evalúa la reforma d...,None,https://perfilesla.flacso.edu.mx/index.php/per...,https://perfilesla.flacso.edu.mx/index.php/per...
4,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",La diversidad institucional del bicameralismo ...,El presente trabajo tiene como foco el estudio...,None,https://perfilesla.flacso.edu.mx/index.php/per...,https://perfilesla.flacso.edu.mx/index.php/per...
...,...,...,...,...,...,...,...,...
241,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",Reclutamiento y profesionalización política: r...,El concepto de representación política llega a...,None,https://perfilesla.flacso.edu.mx/index.php/per...,https://perfilesla.flacso.edu.mx/index.php/per...
242,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...","Vida, pasión y ¿muerte? Del seguro social de s...",En este trabajo se analiza el proceso de refor...,None,https://perfilesla.flacso.edu.mx/index.php/per...,https://perfilesla.flacso.edu.mx/index.php/per...
243,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",La Reforma institucional del corporativismo si...,Este artículo examina de manera comparada los ...,None,https://perfilesla.flacso.edu.mx/index.php/per...,https://perfilesla.flacso.edu.mx/index.php/per...
244,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",Alonso Lujambio. Poder legislativo. Gobiernos ...,,None,https://perfilesla.flacso.edu.mx/index.php/per...,https://perfilesla.flacso.edu.mx/index.php/per...


In [27]:
df_catalogo.to_csv('Revista21.01.csv', index=False)

In [28]:
#df=df_catalogo.drop(df_catalogo.index[[124,126,109,127,146,169]])
#df

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...","Plebiscitos, referendos e iniciativas populare...",En este artículo se estudian y ordenan las dif...,None,https://perfilesla.flacso.edu.mx/index.php/per...,https://perfilesla.flacso.edu.mx/index.php/per...
1,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",Mejorando los programas de combate a la pobrez...,Muchos de los programas sociales y la gran may...,None,https://perfilesla.flacso.edu.mx/index.php/per...,https://perfilesla.flacso.edu.mx/index.php/per...
2,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",Rearme y baja percepción de amenaza interestat...,El propósito de este artículo es realizar un a...,None,https://perfilesla.flacso.edu.mx/index.php/per...,https://perfilesla.flacso.edu.mx/index.php/per...
3,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",Evaluación de la reforma del sistema de pensio...,En el presente artículo se evalúa la reforma d...,None,https://perfilesla.flacso.edu.mx/index.php/per...,https://perfilesla.flacso.edu.mx/index.php/per...
4,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",La diversidad institucional del bicameralismo ...,El presente trabajo tiene como foco el estudio...,None,https://perfilesla.flacso.edu.mx/index.php/per...,https://perfilesla.flacso.edu.mx/index.php/per...
...,...,...,...,...,...,...,...,...
241,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",Reclutamiento y profesionalización política: r...,El concepto de representación política llega a...,None,https://perfilesla.flacso.edu.mx/index.php/per...,https://perfilesla.flacso.edu.mx/index.php/per...
242,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...","Vida, pasión y ¿muerte? Del seguro social de s...",En este trabajo se analiza el proceso de refor...,None,https://perfilesla.flacso.edu.mx/index.php/per...,https://perfilesla.flacso.edu.mx/index.php/per...
243,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",La Reforma institucional del corporativismo si...,Este artículo examina de manera comparada los ...,None,https://perfilesla.flacso.edu.mx/index.php/per...,https://perfilesla.flacso.edu.mx/index.php/per...
244,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",Alonso Lujambio. Poder legislativo. Gobiernos ...,,None,https://perfilesla.flacso.edu.mx/index.php/per...,https://perfilesla.flacso.edu.mx/index.php/per...


In [29]:
#df.to_csv('Revista21.02.csv', index=False)